In [6]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.insert(0, "../")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
from tqdm.auto import tqdm
import torch
import json
from pathlib import Path
from nohomers.ml.lightweight_gan_trainer import get_trainer, gen_images_and_manifest, gen_interpolation_videos
from nohomers.ml.cleaner import load_cleaner
from uuid import uuid4
from PIL import Image
import torchvision
import tempfile
from torchvision import transforms
import numpy as np
import ffmpeg
import shutil
import copy
import random

In [9]:
trainer = get_trainer(
    models_dir="/mnt/evo/projects/metapedia/tmp/stylegan2/models", 
    name="simpsons_large_cleaned_nobackground_1024_augnormal04",
    load_from=65,
).to("cuda:0")

loading from version 0.12.4


In [10]:
cleaner = load_cleaner(Path("/mnt/evo/projects/nohomers/cleaners/simpsons_large_cleaned_nobackground_1024_augnormal04/cleaner.pkl"))

In [36]:
output_dir = Path("/mnt/evo/projects/nohomers/assets/simpsons_large_cleaned_nobackground_1024_augnormal04")
shutil.rmtree(output_dir, ignore_errors=True, onerror=None)
output_dir.mkdir(parents=True, exist_ok=True)

manifest = gen_images_and_manifest(
    trainer, 
    output_dir,
    num=10_000,
    batch_size=100,
    cleaner=cleaner,
    clean_threshold=0.55,
)

In [57]:
video_manifest = gen_interpolation_videos(
    trainer, manifest, output_dir, per_edge=2, video_duration=1.5,
)

In [61]:
manifest_path = Path(output_dir) / "manifest.json"
with open(manifest_path, "w") as f:
    json.dump([e.to_dict() for e in video_manifest], f)

In [24]:
test = generate_interpolation_frames(
    trainer, 
    latents_low=image_and_latents[0][0].unsqueeze(0),
    latents_high=image_and_latents[1][0].unsqueeze(0),
    num_frames=120,
)[0]

In [169]:
frames_to_video(test, output_dir / "test.mp4", fps=30, bitrate="0.5M")

In [160]:
print((output_dir / "test.mp4").exists())

True
